In [27]:
# data fields to download

# Capacity	Fuel	Bioenergy	GW
# Capacity	Fuel	Coal	GW
# Capacity	Fuel	Gas	GW
# Capacity	Fuel	Hydro	GW
# Capacity	Fuel	Nuclear	GW
# Capacity	Fuel	Other Fossil	GW
# Capacity	Fuel	Other Renewables	GW
# Capacity	Fuel	Solar	GW
# Capacity	Fuel	Wind	GW
# Electricity generation	Fuel	Bioenergy	%
# Electricity generation	Fuel	Coal	%
# Electricity generation	Fuel	Gas	%
# Electricity generation	Fuel	Hydro	%
# Electricity generation	Fuel	Nuclear	%
# Electricity generation	Fuel	Other Fossil	%
# Electricity generation	Fuel	Other Renewables	%
# Electricity generation	Fuel	Solar	%
# Electricity generation	Fuel	Wind	%
# Electricity generation	Fuel	Bioenergy	TWh
# Electricity generation	Fuel	Coal	TWh
# Electricity generation	Fuel	Gas	TWh
# Electricity generation	Fuel	Hydro	TWh
# Electricity generation	Fuel	Nuclear	TWh
# Electricity generation	Fuel	Other Fossil	TWh
# Electricity generation	Fuel	Other Renewables	TWh
# Electricity generation	Fuel	Solar	TWh
# Electricity generation	Fuel	Wind	TWh
# Electricity generation	Total	Total Generation	TWh
# Electricity imports	Electricity imports	Net Imports	TWh
# Electricity demand	Demand	Demand	TWh
# Electricity demand	Demand per capita	Demand per capita	MWh


In [309]:
import pandas as pd
import numpy as np # need it to create a logarithmic scale for the geographic map
import plotly_express as px

geo_list = ['China', 'India', 'Indonesia', 'South Africa'] # select cuntries or regions (e.g. 'Europe' and/or 'World') for which to see the data for
data_fields = [
    ['Capacity', 'Fuel', 'GW', 'Capacity by fuel type (GW)'],
    ['Electricity generation', 'Fuel', 'TWh', 'Power generation by fuel type (TWh)'],
    ['Electricity imports', 'Electricity imports', 'TWh', 'Net power imports (TWh)'],
    ['Electricity demand', 'Demand', 'TWh', 'Power demand (TWh)']
    ] # creates list of lists. Each list represents a specific data fueld and its description

def bar_chart_data_func(df, data_fields): # formats and aggregates yearly (i.e. time series) data for selected countries/regions. Returns result as a DataFrame
    df = df[(df['Category']==data_fields[0]) & (df['Subcategory']==data_fields[1]) & (df['Unit']==data_fields[2]) & (df['Area'].isin(geo_list))] # filter rows where values under 'Category' column equal first value in 'data_fields' list (e.g. 'Capacity'), where values under 'Subcategory' column equal second value in 'data_fields' list  (e.g. 'Fuel'), where values under 'Unit' column equal third value in 'data_fields' list  and where 'Area' matches a list of countries provided in 'geo_list' variable
    df = df.groupby(['Variable', 'Year'], group_keys=True, as_index=False)[['Value']].sum() # group fuel types 'Variables' and show it as total sum for selected countries in the 'geo_list'
    return df

def agg_bar_chart_data_func(df, data_fields): # aggregates yearly (i.e. time series) data by fuel type (e.g. Capacity for Wind+Solar+Nuclear) into single value (e.g. total capacity) and returns result as a DataFrame
    df = df.groupby('Year', as_index=False).sum() # aggregates data by fuel type into single value. In the process, removes 'Variable' column
    df['Variable'] = data_fields[0] # adds back 'Variable' column
    return df

def pct_chg_data_func(df): # calculates yearly (i.e. time series) % YoY change time series values based on data calculated using 'bar_chart_data_func()' function and returns result as a DataFrame
    df = df.pivot(index='Year', columns='Variable', values='Value') # moves 'Variable' names to individual columns
    df = df.pct_change()*100 # calculates yearly percentage change
    df = df[1:].reset_index() # removes first row with NaN values and moves 'Year' variable from being index to an individual 'Year' column
    df = df.melt(id_vars=['Year'], value_vars=list(df.columns)[1:], var_name='Variable', value_name='Value') # moves 'Variable' names back to a single column
    return df

def geo_map_data_func(df, data_fields): # formats and aggregates data for all countries for the most current year (i.e. snapshot). Returns result as a DataFrame, to be used later to plot on a geographic map.
    # save ISO3 country codes per country in a DataFrame. ISO3 codes will be used later to create geographical map chart
    df.dropna(subset=['Country code'], inplace=True) # delete all blan (i.e. NaN) entries from 'Country code' column.
    country_names  = list(df['Area'].unique()) # assign unique country names to country_names list
    country_codes = list(df['Country code'].unique()) # assign unique ISO3 country codes to country_codes list
    df_ISO_codes = pd.DataFrame({'Area': country_names, 'Country code': country_codes}) # create 'df_ISO_codes' DataFrame to host country names and their ISO3 codes
    # formats and aggregates data
    df = df[(df['Category']==data_fields[0]) & (df['Subcategory']==data_fields[1]) & (df['Unit']==data_fields[2])] # filter rows where values under 'Category' column equal first value in 'data_fields' list (e.g. 'Capacity'), where values under 'Subcategory' column equal second value in 'data_fields' list  (e.g. 'Fuel'), where values under 'Unit' column equal third value in 'data_fields' list. Downloads data for all countries (but not regions) under the 'Area' column and not just the countries provided in 'geo_list' variable
    df = df.groupby(['Area', 'Year'], group_keys=True, as_index=False)[['Value']].sum() # group total yearly values per country in 'Area' column and show it as total sum for selected countries in the 'geo_list'
    df = pd.merge(df, df_ISO_codes, on ='Area', how ='left').dropna() # add 'ISO3 Code' column to 'df' DataFrame. This will be used later to create a geographic map chart. Also, uses 'dropna()' function to remove NaN values to prevent geographic chart from showing wrong colours due to NaN values.
    df = df.sort_values('Year').groupby('Area').tail(1) # filter to show the latest (by 'Year' column) value per country
    df = df.sort_values('Value') # sort by values
    return df

def geo_bar_chart_data_func(df, geo_list): # formats and aggregates data for selected countries for the most recent year (i.e. snapshot). Returns result as a DataFrame, to be used later to plot on a bar chart.
    df = df[df['Area'].isin(geo_list)]
    return df

def bar_chart_func(df, data_fields, geo_list): # creates yearly (i.e. time series) aggregated bar chart for actual data for countries/regions selected in 'geo_list' variable.
    fig = px.bar(df, 
        x=df['Year'],
        y=df['Value'],
        color=df['Variable'],
        height=500, 
        width=900,
        title=data_fields[3] + ': ' + ' '.join(geo_list),
        # barmode = 'group',
        template='plotly_dark'
        )  # charts the data
    fig.show() # chart aggregated data for selected countries/regions

def pct_chg_line_chart_func(df, data_fields, geo_list): # creates yearly (i.e. time series) bar chart for % YoY change data for countries/regions selected in 'geo_list' variable
    fig = px.line(df, 
        x=df['Year'],
        y=df['Value'],
        color=df['Variable'],
        height=500, 
        width=900,
        title=data_fields[3] + ' (% Chg YoY): ' + ' '.join(geo_list),
        # barmode = 'group',
        template='plotly_dark'
        )  # charts the data
    fig.add_hline(y=df['Value'].describe()[1], line_dash="dash") # adds a vertical average dash line to the chart
    fig.show() # chart aggregated data for selected countries

def geo_map_func(df, data_fields): # creates a geographic map chart for actual data for all countries for the most recent year (i.e. snapshot)
    map_fig = px.choropleth(df, # TODO replace this with 'map_fig = px.choropleth_mapbox(...)' and apply "mapbox_style='carto-positron'" property in order to display country names on a map
                            locations='Country code',
                            projection='orthographic',
                            color=np.log10(df['Value']), # BUG for net imports, sometimes shows in tooltip values for 'Value' field but 'null' for 'color' field.
                            height=500, 
                            width=900,
                            title=data_fields[3] + ': All countries',
                            color_continuous_scale=['green', 'white', 'red'],
                            #color_continuous_scale=px.colors.sequential.Viridis,
                            #color_continuous_scale='Bluered_r',
                            hover_name='Area',
                            hover_data=['Area', 'Value']
    )
    map_fig.show() # plots data for selected countries on a map

def geo_bar_chart_func(df, data_fields, geo_list): # creates bar chart for actual data for countries/regions selected in 'geo_list' variable for the most recent year (i.e. snapshot)
    fig = px.bar(df, 
        x=df['Area'],
        y=df['Value'],
        #color=df['Variable'],
        height=500, 
        width=900,
        title=data_fields[3] + ': ' + ' '.join(geo_list),
        # barmode = 'group',
        template='plotly_dark'
        )  # charts the data
    fig.show() # chart aggregated data for selected countries/regions



df = pd.read_csv('yearly_full_release_long_format-1.csv') # copy content of csv file downloaded from 'https://ember-climate.org/app/uploads/2022/07/yearly_full_release_long_format-1.csv' to 'df' DataFrame
df.drop(['Ember region','EU', 'OECD', 'G20', 'G7', 'YoY absolute change', 'YoY % change'], axis=1, inplace=True) # remove unwanted columns

# run functions

for x in range(len(data_fields)): # loops through list of lists in 'data_fields' column to format and chart the data for each of those lists
    bar_chart_data = bar_chart_data_func(df, data_fields[x]) # function to format and aggregate yearly (i.e. time series) data for selected countries/regions. Then, this function returns result as a DataFrame
    bar_chart = bar_chart_func(bar_chart_data, data_fields[x], geo_list) # function to create a bar chart for yearly (i.e. time series) actual data for countries/regions selected in 'geo_list' variable
    if data_fields[x][1] == 'Fuel': # this condition helps avoid creating duplicate charts for data fields where there is no breakdown by fuel type (e.g. 'Electricity demand' and 'Electricity imports')
        agg_bar_chart_data = agg_bar_chart_data_func(bar_chart_data, data_fields[x]) # function to aggregate yearly (i.e. time series) data by fuel type (e.g. Capacity for Wind+Solar+Nuclear) into single value (e.g. total capacity) and returns result as a DataFrame
        agg_bar_chart = bar_chart_func(agg_bar_chart_data, data_fields[x], geo_list) # function to create a bar chart for aggregated yearly (i.e. time series) actual data for countries/regions selected in 'geo_list' variable
    pct_chg_data = pct_chg_data_func(bar_chart_data) # function to calculate yearly (i.e. time series) % YoY change values based on data calculated previously using 'bar_chart_data_func()' function. Then, this function returns result as a DataFrame
    pct_chg_chart = pct_chg_line_chart_func(pct_chg_data, data_fields[x], geo_list) # function to create a bar chart for yearly (i.e. time series) % YoY change data for countries/regions selected in 'geo_list' variable
    if data_fields[x][1] == 'Fuel': # this condition helps avoid creating duplicate charts for data fields where there is no breakdown by fuel type (e.g. 'Electricity demand' and 'Electricity imports')
        agg_pct_chg_data = pct_chg_data_func(agg_bar_chart_data) # function to calculate yearly (i.e. time series) % YoY change values based on data calculated previously using 'bar_chart_data_func()' function. Then, this function returns result as a DataFrame
        agg_pct_chg_chart = pct_chg_line_chart_func(agg_pct_chg_data, data_fields[x], geo_list) # function to create a bar chart for yearly (i.e. time series) % YoY change data for countries/regions selected in 'geo_list' variable
    geo_map_data = geo_map_data_func(df, data_fields[x]) # function to format and aggregate data for all countries for the most current year (i.e. snapshot). Returns result as a DataFrame, to be used later to plot on a geographic map.
    geo_map = geo_map_func(geo_map_data, data_fields[x]) # function to create a geographic map chart for actual data for all countries for the most recent year (i.e. snapshot)
    geo_bar_chart_data = geo_bar_chart_data_func(geo_map_data, geo_list) # function to format and aggregate data for selected countries for the most recent year (i.e. snapshot). Returns result as a DataFrame, to be used later to plot on a bar chart.
    geo_bar_chart = geo_bar_chart_func(geo_bar_chart_data, data_fields[x], geo_list) # function to create a bar chart for actual data for countries/regions selected in 'geo_list' variable for the most recent year (i.e. snapshot)

c:\Python310\lib\site-packages\pandas\core\arraylike.py:364: RuntimeWarning:

divide by zero encountered in log10



TypeError: choropleth() got an unexpected keyword argument 'opacity'